With this notebook I am creating the transductive train, validation, and test sets. At the beginning I find the trace ids that do not belong to duplicate calls (traces ids that can identify an unique path from an user request). Then I create a transductive train set that has all the entities in the graph. From the remaining dataframe, I remove all the relations that belong to a trace that is

In [1]:
import pandas as pd
import numpy as np

seed = 1234

In [15]:
df = pd.read_csv('MSCallGraph_0.csv').set_index('Unnamed: 0')
df_2 = pd.read_csv('MSCallGraph_1.csv')
df_2['Unnamed: 0'] = df_2['Unnamed: 0']+len(df) #required, otherwise there would be multiple indexes
df_2 = df_2.set_index('Unnamed: 0')
df_2

,traceid,timestamp,rpcid,um,rpctype,dm,interface,rt
Unnamed: 0,,,,,,,,
6088846,0b51067815919239159198000e96b7,316068,0.1.1.2.7.1.26.2.5.1.10,5ba2be786dc249dc3b3cda596a4d121f55a1dcb75b399d...,mc,1ba37accfe145eb0ac812432bad415d1af09aa9cc7998c...,NaN,0
6088847,0b51067815919239159198000e96b7,316068,0.1.1.2.7.1.26.2.5.1.19,5ba2be786dc249dc3b3cda596a4d121f55a1dcb75b399d...,mc,1ba37accfe145eb0ac812432bad415d1af09aa9cc7998c...,NaN,0
6088848,0b51067815919239159198000e96b7,316068,0.1.1.2.7.1.26.2.5.1.18,5ba2be786dc249dc3b3cda596a4d121f55a1dcb75b399d...,mc,1ba37accfe145eb0ac812432bad415d1af09aa9cc7998c...,NaN,0
6088849,0b51067815919239159198000e96b7,316068,0.1.1.2.7.1.26.2.5.1.17,5ba2be786dc249dc3b3cda596a4d121f55a1dcb75b399d...,mc,1ba37accfe145eb0ac812432bad415d1af09aa9cc7998c...,NaN,0
6088850,0b51067815919239159198000e96b7,316068,0.1.1.2.7.1.26.2.5.1.16,5ba2be786dc249dc3b3cda596a4d121f55a1dcb75b399d...,mc,1ba37accfe145eb0ac812432bad415d1af09aa9cc7998c...,NaN,0
...,...,...,...,...,...,...,...,...
12486629,0b14140415919239334367000e783f,333438,0.1.1,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,mc,0b0c981da8042a436f30f7d9f7cce74360abe630bd5478...,NaN,0
12486630,0b14140415919239334367000e783f,333438,0.1,(?),http,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,6e3e36ad3820a2f4dc15760c74fcd53448db5091fa1cdc...,-8
12486631,0b14140415919239334367000e783f,333444,0.1.3.1,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc,643bce6326e327bb36361c31d1ad0cf9ee8d5767175482...,7a12db103eac1dedbe029423beab8c6d2b9351460db1ca...,2


In [16]:
df = pd.concat([df,df_2])
df

,traceid,timestamp,rpcid,um,rpctype,dm,interface,rt
Unnamed: 0,,,,,,,,
0,0b133c1915919238193454000e5d37,219678,0.1.3.1.1.1.12,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,0
1,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.19,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
2,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.18,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
3,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.4,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
4,0b133c1915919238193454000e5d37,219684,0.1.3.1.1.14.17,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...,NaN,1
...,...,...,...,...,...,...,...,...
12486629,0b14140415919239334367000e783f,333438,0.1.1,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,mc,0b0c981da8042a436f30f7d9f7cce74360abe630bd5478...,NaN,0
12486630,0b14140415919239334367000e783f,333438,0.1,(?),http,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,6e3e36ad3820a2f4dc15760c74fcd53448db5091fa1cdc...,-8
12486631,0b14140415919239334367000e783f,333444,0.1.3.1,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc,643bce6326e327bb36361c31d1ad0cf9ee8d5767175482...,7a12db103eac1dedbe029423beab8c6d2b9351460db1ca...,2


In [17]:
df.nunique()

traceid       262722
timestamp     599837
rpcid        1288588
um              1777
rpctype            6
dm              8757
interface       7529
rt              3746
dtype: int64

In [19]:
df=df.drop(['timestamp','rpcid','interface','rt',],axis=1)
df = df.replace('(?)', np.NaN)
df = df.replace('', np.NaN)
df = df.replace('NAN', np.NaN)
df = df.dropna()
df = df.drop_duplicates()
df.nunique()

traceid    262717
um           1766
rpctype         6
dm           8716
dtype: int64

In [20]:
df

,traceid,um,rpctype,dm
Unnamed: 0,,,,
0,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
11,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
20,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
26,0b133c1915919238193454000e5d37,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
42,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...,...
12486627,0b1413aa15919241962278000e94a9,b58ba81e712e15281eb93074ed026a844f5100024d91b2...,mc,0b0c981da8042a436f30f7d9f7cce74360abe630bd5478...
12486629,0b14140415919239334367000e783f,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,mc,0b0c981da8042a436f30f7d9f7cce74360abe630bd5478...
12486631,0b14140415919239334367000e783f,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc,643bce6326e327bb36361c31d1ad0cf9ee8d5767175482...


In [21]:
traces = list(set(df['traceid']))
unique_trace_ids = []
triplets_df = df.drop(['traceid'], axis = 1)
triplets_df = triplets_df.drop_duplicates()
triplets_df

,um,rpctype,dm
Unnamed: 0,,,
0,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
11,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
20,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
26,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
42,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...
12481024,5aefad73e20ae83cd0bf473107dd290e74f9a8b12aa6bc...,mq,04629083dca233713ffc43c844ebeddc780373190c10fa...
12481035,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,db,5d6e5a8ded9588bd6462250d74d5a5fd8f069519e47dc1...
12481595,d03bb97862607468fe3153b28d41a20de1e3144a566264...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...


In [22]:
filtered_df = df.loc[triplets_df.index]
filtered_df

,traceid,um,rpctype,dm
Unnamed: 0,,,,
0,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
11,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
20,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
26,0b133c1915919238193454000e5d37,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
42,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...,...
12481024,0b017c0315919205566434000e47df,5aefad73e20ae83cd0bf473107dd290e74f9a8b12aa6bc...,mq,04629083dca233713ffc43c844ebeddc780373190c10fa...
12481035,0b01d9d115919241822149000d06d1,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,db,5d6e5a8ded9588bd6462250d74d5a5fd8f069519e47dc1...
12481595,0b01d9d115919241822149000d06d1,d03bb97862607468fe3153b28d41a20de1e3144a566264...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...


In [23]:
filtered_traces =list(set(filtered_df['traceid']))
len(filtered_traces) # theoretical maximum nuber of unique traces

5750

In [24]:
i = 0
import time
import IPython
tot_n_traces = len(filtered_traces)
tot_n_triplets = len(triplets_df)
start = time.time()
for trace in filtered_traces:
    trace_calls_df = df[df['traceid'] == trace]
    trace_calls_df = trace_calls_df.drop(['traceid'],axis = 1)
    trace_calls_df = trace_calls_df.drop_duplicates()
    
    intersection_df = pd.merge(triplets_df, trace_calls_df)
    if len(intersection_df) > 0:
        unique_trace_ids.append(trace)
        # intersection is a subset of triplets
        triplets_df = pd.concat([triplets_df,intersection_df]).drop_duplicates(keep=False)
    
    remaining_triplets = len(triplets_df)
    if remaining_triplets == 0:
        break
    
    i = i+1
    duration = time.time() - start
    remaining_time = duration/(i/tot_n_traces) - duration
    print("% traces completed: ", (i/tot_n_traces) * 100)
    print("Traces remaining time (s): ", remaining_time)
    print("Traces remaining time (m): ", remaining_time/60)
    print("Traces remaining time (h): ", remaining_time/60/60)
    print("Traces remaining time (d): ", remaining_time/60/60/24)
    
    remaining_time_triplets = duration/(1-(remaining_triplets/tot_n_triplets)) - duration
    print("Remaining triplets: ", remaining_triplets)
    print("% triplets completed: ", (1-(remaining_triplets/tot_n_triplets)) *100)
    print("Triplets remaining time (s): ", remaining_time_triplets)
    print("Triplets remaining time (m): ", remaining_time_triplets/60)
    print("Triplets remaining time (h): ", remaining_time_triplets/60/60)
    print("Triplets remaining time (d): ", remaining_time_triplets/60/60/24)
    IPython.display.clear_output(wait = True)

% traces completed:  99.98260869565217
Traces remaining time (s):  0.18972344526218876
Traces remaining time (m):  0.0031620574210364794
Traces remaining time (h):  5.270095701727466e-05
Traces remaining time (d):  2.195873209053111e-06
Remaining triplets:  1
% triplets completed:  99.99511050264033
Triplets remaining time (s):  0.05333333757835135
Triplets remaining time (m):  0.0008888889596391891
Triplets remaining time (h):  1.4814815993986484e-05
Triplets remaining time (d):  6.172839997494368e-07


In [25]:
duration = time.time() - start
print("Time required (s): ",duration)

Time required (s):  1090.929084777832


In [26]:
print("Number of unique microservice calls: ",len(unique_trace_ids))

Number of unique microservice calls:  4424


In [27]:
unique_calls_df = df[df['traceid'].isin(unique_trace_ids)]
unique_calls_df = unique_calls_df.drop_duplicates()

In [28]:
unique_calls_df.to_csv('unique_calls_df_0_1.csv',index=False)


In [29]:
unique_calls_df = pd.read_csv('unique_calls_df_0_1.csv')
unique_calls_triplets_df = unique_calls_df.drop(['traceid'],axis = 1)
unique_calls_triplets_df = unique_calls_triplets_df.drop_duplicates()

In [30]:
unique_calls_df

,traceid,um,rpctype,dm
0,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
1,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
2,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
3,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
4,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...
...,...,...,...,...
301852,0b01d9d115919241822149000d06d1,7634eabb934a9c9504b24483344242f6e060f8599aa48d...,mq,72b69d6cf010e618d6079185f9ad7dbfb01f2037526ac3...
301853,0b01d9d115919241822149000d06d1,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,mc,aad206b88466844ad1d958b5f1fadddfee89d1e304b55b...
301854,0b01d9d115919241822149000d06d1,0e196e772ff517533a7396537ab970729fbbc9ae2f5685...,mc,ab349b930756ee2ec25c9a4ac09c30e075fb5978c83fe0...
301855,0b01d9d115919241822149000d06d1,57ee5b45172bf412c66c61a0ea3bc4b4b42573d4f214e8...,mq,8696ac273534e9ce5f5765029c80c6deeaf539977e1813...


In [31]:
unique_calls_triplets_df

,um,rpctype,dm
0,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
1,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
2,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
4,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...
...,...,...,...
301562,5aefad73e20ae83cd0bf473107dd290e74f9a8b12aa6bc...,mq,04629083dca233713ffc43c844ebeddc780373190c10fa...
301567,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,db,5d6e5a8ded9588bd6462250d74d5a5fd8f069519e47dc1...
301781,d03bb97862607468fe3153b28d41a20de1e3144a566264...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...
301787,f10612cc80a4777bcb0f9025041b3e4a9d8de8b20bfd2d...,db,5d6e5a8ded9588bd6462250d74d5a5fd8f069519e47dc1...


In [32]:
df_copy = unique_calls_df
df = df.drop(['traceid'],axis = 1)
df = df.drop_duplicates()
df

,um,rpctype,dm
Unnamed: 0,,,
0,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
11,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
20,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
26,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
42,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...
12481024,5aefad73e20ae83cd0bf473107dd290e74f9a8b12aa6bc...,mq,04629083dca233713ffc43c844ebeddc780373190c10fa...
12481035,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,db,5d6e5a8ded9588bd6462250d74d5a5fd8f069519e47dc1...
12481595,d03bb97862607468fe3153b28d41a20de1e3144a566264...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...


Checking that all unique triplets are present

In [33]:
assert len(unique_calls_triplets_df) == len(df)

In [34]:
df = unique_calls_triplets_df

In [35]:
entities = set(df['dm']) | set(df['um'])
len(entities)

8791

In [36]:
baseline = []
for entity in entities:
    baseline.append(df[df['dm'] == entity].head(1))
    baseline.append(df[df['um'] == entity].head(1))

In [37]:
baseline_df = pd.concat(baseline)
baseline_df = baseline_df.drop_duplicates()
baseline_df

,um,rpctype,dm
18235,48c0f8f58276081b308fa141cd65a0b0b075a96fcd8822...,db,8911b9efc632ef888c6e803d69f7bd6e4ff30a1e249c22...
23741,c921bfb1f34984199631b16d3e58d1e02fb9e35d3ca97c...,mq,3e2bacc2d4ade304aeb1ba08ed5cb18378e250471bc2e7...
225665,3e2bacc2d4ade304aeb1ba08ed5cb18378e250471bc2e7...,mc,6e85756dd227e486261bf1b4619d05a8e66a2e81756f0e...
30008,af42b5e3e0eb334d38619733586d78d1414f6549f24d31...,db,5c57088d89a55487128b7e66ca0811cb6b4b949f992b11...
85757,da5ac969e69381adca65851753b9ed2decfe4509420484...,mq,581734a302c7672a94ce5dfea1c1fd552332f2bdb785df...
...,...,...,...
278244,17b2aa2ff8ca1795f1f3923682ffc1a646ed32bb9db06e...,rpc,8362863697642ae8fce605157bd9966b0218c48b0f7e7c...
38102,ea2870f1d8a69a4a7a108ac6a56566994ee1359f857eae...,rpc,8182d457578bb18d3f24346c45921fbd642cb9bbcc5d14...
38103,8182d457578bb18d3f24346c45921fbd642cb9bbcc5d14...,rpc,b5cebafb32f9d9f92a3458b781d721e18fdf2de674a3ac...
51975,804eb0d44ed9f1ade6bdb61113d3390ae28b226eeab8a6...,rpc,9775e452ba9c836dce0aab524b9c05b6b4b80cdfcffe72...


In [38]:
baseline_traces_df = df_copy.loc[baseline_df.index]
baseline_traces_df

,traceid,um,rpctype,dm
18235,0b51067115919236138586000eba32,48c0f8f58276081b308fa141cd65a0b0b075a96fcd8822...,db,8911b9efc632ef888c6e803d69f7bd6e4ff30a1e249c22...
23741,0b51144815919236977256000efb7f,c921bfb1f34984199631b16d3e58d1e02fb9e35d3ca97c...,mq,3e2bacc2d4ade304aeb1ba08ed5cb18378e250471bc2e7...
225665,0b5218ef15919239970593000e2b24,3e2bacc2d4ade304aeb1ba08ed5cb18378e250471bc2e7...,mc,6e85756dd227e486261bf1b4619d05a8e66a2e81756f0e...
30008,0b51174c15919238617899000e6ebf,af42b5e3e0eb334d38619733586d78d1414f6549f24d31...,db,5c57088d89a55487128b7e66ca0811cb6b4b949f992b11...
85757,0b51193d15919238080311000e9dde,da5ac969e69381adca65851753b9ed2decfe4509420484...,mq,581734a302c7672a94ce5dfea1c1fd552332f2bdb785df...
...,...,...,...,...
278244,0b8b2cdb15919239623124000d070b,17b2aa2ff8ca1795f1f3923682ffc1a646ed32bb9db06e...,rpc,8362863697642ae8fce605157bd9966b0218c48b0f7e7c...
38102,0b51183f15919237674064000e7dcc,ea2870f1d8a69a4a7a108ac6a56566994ee1359f857eae...,rpc,8182d457578bb18d3f24346c45921fbd642cb9bbcc5d14...
38103,0b51183f15919237674064000e7dcc,8182d457578bb18d3f24346c45921fbd642cb9bbcc5d14...,rpc,b5cebafb32f9d9f92a3458b781d721e18fdf2de674a3ac...
51975,0b01e9a315919237202545000e12c0,804eb0d44ed9f1ade6bdb61113d3390ae28b226eeab8a6...,rpc,9775e452ba9c836dce0aab524b9c05b6b4b80cdfcffe72...


In [39]:
training_traces = set(baseline_traces_df['traceid'])
len(training_traces)

2993

In [40]:
total_train_traces_df = unique_calls_df[unique_calls_df['traceid'].isin(training_traces)]
total_train_traces_df

,traceid,um,rpctype,dm
0,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
1,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
2,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
3,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
4,0b1339ef15919237750706000eded3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...
...,...,...,...,...
301549,0b52060f15919240298388000ef9fe,da5ac969e69381adca65851753b9ed2decfe4509420484...,mq,2be3b98bfce0e36bfa57f93d594bb1e183b434540c5bce...
301550,0b52060f15919240298388000ef9fe,2be3b98bfce0e36bfa57f93d594bb1e183b434540c5bce...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...
301551,0b52060f15919240298388000ef9fe,da5ac969e69381adca65851753b9ed2decfe4509420484...,mq,a499765ef7294335d7a0280bb5dd98c671e58409d953f6...
301552,0b52060f15919240298388000ef9fe,803b003f7c18e5f655b75264079645c9120aba02a3672b...,mq,da5ac969e69381adca65851753b9ed2decfe4509420484...


In [41]:
train_triplets_df = total_train_traces_df.drop(['traceid'],axis = 1).drop_duplicates()
# train_triplets_df = baseline_traces_df
train_triplets_df

,um,rpctype,dm
0,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
1,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
2,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
4,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...
...,...,...,...
301410,58d745e6fac32dd9ddb2db4a491e3d5deed19311aa0109...,mq,a2a88e609089c0df6d5b7a458bb1af295bfc0af5fbb6b7...
301424,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,db,220795bc3a6860e031bb83cab9301b636a3c3bb15e6b80...
301482,7a481b9a132c57cae45f19d82be7b758f789203f2c796c...,db,9141456ba0aa0e7db6a5ae0b2b0f38c63f4039629faaa1...
301516,58d745e6fac32dd9ddb2db4a491e3d5deed19311aa0109...,rpc,4279d89d4b4237786f37f9999ef43b0bc9fa6195b844f5...


In [42]:
# the traces in the training set may not be completed
# train_df = baseline_df
train_df = train_triplets_df
train_df

,um,rpctype,dm
0,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
1,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
2,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
4,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...
...,...,...,...
301410,58d745e6fac32dd9ddb2db4a491e3d5deed19311aa0109...,mq,a2a88e609089c0df6d5b7a458bb1af295bfc0af5fbb6b7...
301424,12bc3fe2791d9f2946a74114ff349795461e695a099ae3...,db,220795bc3a6860e031bb83cab9301b636a3c3bb15e6b80...
301482,7a481b9a132c57cae45f19d82be7b758f789203f2c796c...,db,9141456ba0aa0e7db6a5ae0b2b0f38c63f4039629faaa1...
301516,58d745e6fac32dd9ddb2db4a491e3d5deed19311aa0109...,rpc,4279d89d4b4237786f37f9999ef43b0bc9fa6195b844f5...


In [43]:
triplets_not_present_in_training = pd.concat([df,train_triplets_df]).drop_duplicates(keep = False)
traces_with_triplets_used_in_training_df = unique_calls_df.drop(triplets_not_present_in_training.index)
traces_with_triplets_used_in_training = set(traces_with_triplets_used_in_training_df['traceid'])
not_used_triplets_df = unique_calls_df[~unique_calls_df['traceid'].isin(traces_with_triplets_used_in_training)]
# not_used_triplets_df = unique_calls_df.drop(unique_calls_df.drop(pd.concat([unique_calls_df.drop(['traceid'],axis=1).drop_duplicates(),train_triplets_df]).drop_duplicates(keep = False).index).index)
not_used_triplets_df

,traceid,um,rpctype,dm
67564,0b51187a15918992051312000e0a8b,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,1efeaa8c6ee7899fecd33c8b0ed024154adcc03c7899c1...


In [44]:
len(set(not_used_triplets_df['traceid']))

1

In [45]:
not_baseline_df = not_used_triplets_df.drop('traceid',axis=1).drop_duplicates()
not_baseline_df

,um,rpctype,dm
67564,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,1efeaa8c6ee7899fecd33c8b0ed024154adcc03c7899c1...


In [46]:
# training_traces_df = df_copy.loc[train_df.index]
# training_traces_df

In [47]:
# training_traces = set(training_traces_df['traceid'])
# len(training_traces)

In [48]:
# # triplets that have not been used in training
# remaining_df_1 = df_copy[~ df_copy['traceid'].isin(training_traces)].drop(['traceid'],axis = 1).drop_duplicates()
# # triplets of relations that have not been used in training
# remaining_df_2 = df.drop(training_traces_df.index)
# remaining_df = remaining_df_2.merge(remaining_df_1,left_index= True,right_on=remaining_df_1.index, how = 'inner').drop(
#     ['key_0','um_y','rpctype_y','dm_y'],axis = 1).rename(columns={'um_x':'um','rpctype_x':'rpctype','dm_x':'dm'})
remaining_df = not_baseline_df
remaining_df

,um,rpctype,dm
67564,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,1efeaa8c6ee7899fecd33c8b0ed024154adcc03c7899c1...


In [49]:
remaining_traces_df = df_copy.loc[remaining_df.index]
remaining_traces_df

,traceid,um,rpctype,dm
67564,0b51187a15918992051312000e0a8b,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,1efeaa8c6ee7899fecd33c8b0ed024154adcc03c7899c1...


In [50]:
remaining_traces = list(set(remaining_traces_df['traceid']))
len(remaining_traces)

1

In [51]:
valid_testing_df = remaining_df
valid_testing_df

,um,rpctype,dm
67564,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,1efeaa8c6ee7899fecd33c8b0ed024154adcc03c7899c1...


In [52]:
validation_df = valid_testing_df.head(0)
i = 0    # I want the test set to have at least 60% of the remaining traces
while i < len(remaining_traces) and len(valid_testing_df) > len(remaining_df) * 0.54:
    trace = remaining_traces[i]
    temp_df = remaining_traces_df[remaining_traces_df['traceid'] == trace].drop(['traceid'],axis=1).drop_duplicates()
    validation_df = pd.concat([validation_df, temp_df])
    valid_testing_df = valid_testing_df.drop(temp_df.index)
    remaining_traces_df = remaining_traces_df.drop(temp_df.index)
    i = i+1

In [53]:
validation_df

,um,rpctype,dm
67564,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,1efeaa8c6ee7899fecd33c8b0ed024154adcc03c7899c1...


In [54]:
test_df = valid_testing_df
test_df

,um,rpctype,dm


trace leakage checks

In [55]:
train_traces = set(df_copy.loc[train_df.index]['traceid'])
validation_traces = set(df_copy.loc[validation_df.index]['traceid'])
test_traces = set(df_copy.loc[test_df.index]['traceid'])

In [56]:
assert len(train_traces & validation_traces) == 0

In [57]:
assert len(test_traces & validation_traces) == 0

In [58]:
assert len(train_traces & test_traces) == 0

check to see if the traces are completed (entirely contained in the sets)

In [59]:
for trace in train_traces:
    all_triplets_trace_df = unique_calls_df[unique_calls_df['traceid']==trace].drop(['traceid'],axis = 1).drop_duplicates()
    len_total_triplets = len(all_triplets_trace_df)
    len_contained = len(train_df.merge(all_triplets_trace_df))
    assert (len_total_triplets == len_contained),f"{len_total_triplets}  {len_contained}" 

In [60]:
for trace in validation_traces:
    all_triplets_trace_df = unique_calls_df[unique_calls_df['traceid']==trace].drop(['traceid'],axis = 1).drop_duplicates()
    len_total_triplets = len(all_triplets_trace_df)
    len_contained = len(validation_df.merge(all_triplets_trace_df))
    assert (len_total_triplets == len_contained),f"{len_total_triplets}  {len_contained}" 

In [61]:
for trace in test_traces:
    all_triplets_trace_df = unique_calls_df[unique_calls_df['traceid']==trace].drop(['traceid'],axis = 1).drop_duplicates()
    len_total_triplets = len(all_triplets_trace_df)
    len_contained = len(test_df.merge(all_triplets_trace_df))
    assert (len_total_triplets == len_contained),f"{len_total_triplets}  {len_contained}" 

Transductive checks

In [62]:
train_entities = set(train_df['dm']) or set(train_df['um'])
valid_entities = set(validation_df['dm']) or set(validation_df['um'])
test_entities = set(test_df['dm']) or set(test_df['um'])

In [63]:
assert(valid_entities.issubset(train_entities))

In [64]:
assert(test_entities.issubset(train_entities))

In [65]:
train_relations = set(train_df['rpctype'])
valid_relations = set(validation_df['rpctype'])
test_relations = set(test_df['rpctype'])

In [66]:
assert(valid_relations.issubset(train_relations))

In [67]:
assert(test_relations.issubset(train_relations))

Check to see if there is test leakage

In [68]:
assert(0==len(pd.merge(train_df, validation_df)))

In [69]:
assert(0==len(pd.merge(train_df, test_df)))

In [70]:
assert(0==len(pd.merge(test_df, validation_df)))

In [ ]:
def create_triplets(df):
    triplets = []
    for i in range(len(df)):
        head = df.iloc[i]['dm']
        tail = df.iloc[i]['um']
        rel = df.iloc[i]['rpctype']
        triplets.append([head,rel,tail])
    return triplets

In [44]:
import asposecells
import jpype
jpype.startJVM()

from asposecells.api import Workbook, FileFormatType

def create_tsv(triplets, file_name):


    # Create Workbook object.
    workbook = Workbook(FileFormatType.TSV)

    # Access the first worksheet of the workbook.
    worksheet = workbook.getWorksheets().get(0)

    # Get the desired cell(s) of the worksheet and input the value into the cell(s).


    i = 1
    for elem in triplets: 
        worksheet.getCells().get("A"+str(i)).putValue(elem[0])
        worksheet.getCells().get("B"+str(i)).putValue(elem[1])
        worksheet.getCells().get("C"+str(i)).putValue(elem[2])
        i+=1


    # Save the workbook as TSV file.
    workbook.save(file_name)


    file = open(file_name,'r')  
    lines = file.readlines()  
    file.close()
    file = open(file_name,'w')  
    lines = lines[:-1]
    file.writelines(lines)
    file.close()

In [45]:
datasets = [train_df, validation_df, test_df]
names = ["train", "validation","test"]

In [46]:
for elem in zip(datasets,names):
    create_tsv(create_triplets(elem[0]),f"MSCallGraph_0_1_{elem[1]}_transductive_traces.tsv")
jpype.shutdownJVM()